<a href= "https://www.tutorialspoint.com/db2/db2_introduction.htm"><img src = "https://www.logolynx.com/images/logolynx/d6/d62afc1ca321ac4366aba9a2abb32a07.jpeg" width = 250, align= "center"></a>

<h1 align = center><font size = 5>Lab: Access DB2 on Cloud using Python</font></h1>

# INTRODUCTION

This notebook illustrates how to access your database instance using Python by following the steps below:

1. Import the `ibm_db` library
1. Identify and enter the database connection credentials
1. Create the database connection
1. Create a table
1. Insert data into the table
1. Query data from the table
1. Retrieve the result set into a pandas dataframe
1. Close the database connection

**NOTE:** To Create a database service instance of Db2 on Cloud [Click Here](https://cloud.ibm.com/docs/Db2onCloud?topic=Db2onCloud-getting-started)

## Step 1: Import the `ibm_db` Python library

In [2]:
#Load ibm_db library

import ibm_db

When the command above completes, the `ibm_db` library is loaded in your notebook.

## Step 2: Identify the database connection credentials

Connecting to dashDB or DB2 database requires the following information:
* Driver Name
* Database name 
* Host DNS name or IP address 
* Host port
* Connection protocol
* User ID
* User Password



**NOTE:** To obtain credentials please refer to the instructions given [here](https://cloud.ibm.com/docs/Db2onCloud?topic=Db2onCloud-db_details_cxn_creds)

Now enter your database credentials below

Replace the placeholder values in between `""` below with your actual database credentials 

e.g. replace "database" with "BLUDB"

In [ ]:
#Replace the placeholder values with the actuals for your Db2 Service Credentials

dsn_driver = "{IBM DB2 ODBC DRIVER}"
dsn_database = "database"            # e.g. "BLUDB"
dsn_hostname = "hostanme"            # e.g.: "dashdb-txn-sbox-yp-dal09-04.services.dal.bluemix.net"
dsn_port = "port"                    # e.g. "50000" 
dsn_protocol = "protocol"            # i.e. "TCPIP"
dsn_uid = "user id"                  # e.g. "abc12345"
dsn_pwd = "password"                 # e.g. "7dBZ3wWt9XN6$o0J"

## Step 3: Create the database connection

Ibm_db API uses the IBM Data Server Driver for ODBC and CLI APIs to connect to IBM DB2 and Informix.


Create the database connection

In [ ]:
#Create database connection

dsn = (
    "DRIVER={0};"
    "DATABASE={1};"
    "HOSTNAME={2};"
    "PORT={3};"
    "PROTOCOL={4};"
    "UID={5};"
    "PWD={5};").format(dsn_driver, dsn_database, dsn_hostname, dsn_port, dsn_protocol, dsn_uid, dsn_pwd)

try:
    conn = ibm_db.connect(dsn, "", "")
    print("Connected to database: ", dsn_database, "as user: ", dsn_uid, "on host: ", dsn_hostname)
    
except:
    print("Unable to connect: ", ibm_db.conn_errormsg())
    

## Step 4: Create a table in the database

In this step we will create a table in the database with following details:

|  COLUMN NAME  |  DATA TYPE  |  NULLABLE  |
|:-------------:|:-----------:|:----------:|
|EMP_ID         |INTEGER      |N           |
|FNAME          |VARCHAR      |Y           |
|LNAME          |VARCHAR      |Y           |
|D_O_B          |DATE         |Y           |
|GENDER         |CHARACTER    |Y           |
|SALARY         |INTEGER      |Y           |
|CITY           |VARCHAR      |Y           |
|CCODE          |CHARACTER    |Y           |

**NOTE:** Here `EMP_ID` is the primary key.

In [ ]:
#Lets first drop the table EMPLOYEES in case it exists from a previous attempt
dropQuery = "drop table EMPLOYEES"

#Now execute the drop statement
dropStmt = ibm_db.exec_immediate(conn, dropQuery)


## Dont worry if you get this error:
If you see an exception/error similar to the following, indicating that EMPLOYEES is an undefined name, that's okay. It just implies that the EMPLOYEES table does not exist in the table - which would be the case if you had not created it previously.

Exception: [IBM][CLI Driver][DB2/LINUXX8664] SQL0204N  "ABC12345.EMPLOYEES" is an undefined name.  SQLSTATE=42704 SQLCODE=-204

In [ ]:
#Construct the Create Table DDL statement

createQuery = "create table EMPLOYEES (EMP_ID Integer Primary Key Not Null, FNAME varchar(20), LNAME varchar(20), D_O_B date, Gender character(1), Salary integer, City varchar(20), CCode character (2))"

#Now fill in the name of the method and execute the statement

createStmt = ibm_db.exec_immediate(conn, createQuery)

## Step 5: Insert data into the table

In this step we will insert some rows of data into the table. 

|  EMP_ID       |  FNAME      |  LNAME     |  D_O_B    |  Gender   |  Salary   |  City     |  CCode    |
|:-------------:|:-----------:|:----------:|:---------:|:---------:|:---------:|:---------:|:---------:|
|01             |Chris        |Evans       |1981-06-13 | M         | 100000    | Boston    | US        |
|02             |Robert       |Downey Jr   |1965-04-04 | M         | 150000    | Manhattan | US        |
|03             |Scarlett     |Johansson   |1984-11-12 | F         | 110000    | Manhattan | US        |
|04             |Chris        |Hemsworth   |1983-08-11 | M         | 90000     | Melbourne | AU        |


We will be using a single query to insert all rows of data.

In [ ]:
#Construct the query

insertQuery = "insert into EMPLOYEES values (01, 'Chris', 'Evans', '1981-06-13', 'M', 100000, 'Boston', 'US'), (02, 'Robert', 'Downey Jr', '1965-04-04', 'M', 150000, 'Manhattan', 'US'), (03, 'Scarlett', 'Johansson', '1984-11-22', 'F', 110000, 'Manhattan', 'US'), (04, 'Chris', 'Hemsworth', '1983-08-11', 'M', 90000, 'Melbourne', 'AU')"

#execute the statement

insertStmt = ibm_db.exec_immediate(conn, insertQuery)

## Step 6: Query data in the table

In this step we will retrieve data we inserted into the EMPLOYEE table. 

In [ ]:
#Construct the query that retrieves all rows from the EMPLOYEES table
selectQuery = "select * from EMPLOYEES"

#Execute the statement
selectStmt = ibm_db.exec_immediate(conn, selectQuery)

#Fetch the Dictionary (for the first row only)
ibm_db.fetch_both(selectStmt)

In [ ]:
#Fetch the rest of the rows and print the EMP_ID and FNAME for those rows

while ibm_db.fetch_row(selectStmt) != False:
    print("EMP_ID: ", ibm_db.result(selectStmt, 0), "FNAME: ", ibm_db.result(selectStmt, "FNAME")) 

Now, we write and execute an update statement that changes the Scarlett's CITY to Los Angeles.

In [ ]:
#Write and execute an update statement 
#change Scarlett's city to Los Angeles

updateQuery = "update EMPLOYEES set CITY = 'Los Angeles' where FNAME = 'Scarlett'"

updateStmt = ibm_db.exec_immediate(conn, updateQuery) 

## Step 7: Retrieve data into Pandas 

In this step we will retrieve the contents of the EMPLOYEE table into a Pandas dataframe.

In [ ]:
import pandas
import ibm_db_dbi

In [ ]:
#connection for pandas
pconn = ibm_db_dbi.Connection(conn)

In [ ]:
#query statement to retrieve all rows in EMPLOYEES table
selectQuery = "select * from EMPLOYEES"

#retrieve the query results into a pandas dataframe
pdf = pandas.read_sql(selectQuery, pconn)

#print just the LNAME for first row in the pandas data frame
pdf.LNAME[0] 

In [ ]:
#print the entire data frame

pdf 

Once the data is in a Pandas dataframe, you can do the typical pandas operations on it. 

For example, you can use the shape method to see how many rows and columns are in the dataframe

In [ ]:
pdf.shape

## Task 8: Close the Connection
We free all resources by closing the connection.  
**NOTE:** Remember that it is always important to close connections so that we can avoid unused connections taking up resources.


In [ ]:
ibm_db.close(conn)

## <U>Summary</U>

In this tutorial we established a connection to a database instance of DB2 Warehouse on Cloud from a Python notebook using ibm_db API. Then created a table and inserted a few rows of data into it. Then queried the data. We also retrieved the data into a pandas dataframe.  

## THANK YOU!